# Introduction to ASE

[Atomic Simulation Environment (ASE)](https://wiki.fysik.dtu.dk/ase/index.html) is a Python package for simulating and working with atomic systems, and is commonly used in the field of atomic modelling. In this notebook, we give examples of how to perform common tasks using ASE. 

In [1]:
from ase import Atoms
from ase.build import bulk

## The Atoms object

The [`Atoms`](https://wiki.fysik.dtu.dk/ase/ase/atoms.html) object is the central entity in ASE, and each object represents a collection of atoms. This object contains information such as atom species, positions, forces, unit cell and so forth. The `Atoms` object also exposes several functions that acts on the object. Please see the [documentation](https://wiki.fysik.dtu.dk/ase/ase/atoms.html) for details. 

Creating an atoms object representing, for instance, `CO` is straightforward:

In [2]:
co = Atoms('CO', positions=[(0,0,0), (0,0,1.1)])
print(co)
print(f'Symbols: {co.symbols}')
print(f'Positions: {co.get_positions().flatten()}')

Atoms(symbols='CO', pbc=False)
Symbols: CO
Positions: [0.  0.  0.  0.  0.  1.1]


One can also use convenience functions for building e.g. bulk structures. Note that one can, for instance, change the lattice constant of the structure. This is useful in situations where you for instance have a relaxed structure in which the lattice parameters differ from `ASE`'s defaults. 

In [3]:
# The lattice parameter is 'a'
cu_bulk = bulk('Cu', 'fcc', a=3.2)
print(cu_bulk)

Atoms(symbols='Cu', pbc=True, cell=[[0.0, 1.6, 1.6], [1.6, 0.0, 1.6], [1.6, 1.6, 0.0]])


## Working with databases

`ASE` also exposes functionality for saving and loading structures, in the form of an [`ASE Database`](https://wiki.fysik.dtu.dk/ase/ase/db/db.html). The databases are files with the ending `.db`, and they can both be maniupulated via the command line or through Python. Each row in the database corresponds to a single `Atoms` object, in other words, each database entry is a structure.

### Command line usage

To view the contents of a database from the command line, type `ase db abc.db`, where `abc.db` is your database file. One can also optionally add the flag `--c++` to show all the columns in the database. Please see the documentation for more detailed usage of the command line interface. 

### Python usage

Databases can also be manipulated using Python. In this example, we will write three different `CO` structures to a database called `abc.db`, and then show how we can read them back into Python. We will also calculate the potential energy of each structure using a Lennard-Jones potential, implemented trough an `ASE Calculator`, which will be saved as a property `lj_energy` for each structure. How `Calculators` work is outside of the scope of this tutorial, but if you're curious you can find the documentation [here](https://wiki.fysik.dtu.dk/ase/ase/calculators/calculators.html).

To connect to a database, use the `connect` command. Note that if the database doesn't exist it will be created, and that existing databases will be overwritten if the `append`-option to connect is `False`.

In [4]:
from ase.db import connect

db = connect('abc.db', append=False)

In [5]:
from ase.calculators.lj import LennardJones

# Create three different CO structures with different spacing
d = [1.1, 1.5, 1.8]
structures = [Atoms('CO', positions=[(0,0,0), (0, 0, di)], calculator=LennardJones()) for di in d]

print(structures)

[Atoms(symbols='CO', pbc=False, calculator=LennardJones(...)), Atoms(symbols='CO', pbc=False, calculator=LennardJones(...)), Atoms(symbols='CO', pbc=False, calculator=LennardJones(...))]


In [6]:
# Write the list of structures to the database
for structure in structures:
    lj_energy = structure.get_potential_energy()
    db.write(structure, lj_energy=lj_energy)

Now we can read the structures back into Python. Note that we convert a Database entry, a `row`, into an `Atoms` object using the `row.toatoms()` method. Furthermore, we access the `lj_energy` property through `row.lj_energy`.

In [7]:
loaded_structures = []
lj_energies = []
for row in db.select():
    print(row)
    loaded_structures.append(row.toatoms())
    lj_energies.append(row.lj_energy)
    
print(loaded_structures)
print(lj_energies)

<AtomsRow: formula=CO, keys=lj_energy>
<AtomsRow: formula=CO, keys=lj_energy>
<AtomsRow: formula=CO, keys=lj_energy>
[Atoms(symbols='CO', pbc=False, calculator=SinglePointCalculator(...)), Atoms(symbols='CO', pbc=False, calculator=SinglePointCalculator(...)), Atoms(symbols='CO', pbc=False, calculator=SinglePointCalculator(...))]
[-0.9778930076294439, -0.3148571525343358, -0.10866761384238112]
